In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key="api key",
    base_url="https://api.openai.com/v1"
)


In [ ]:
import os

#print("OPENAI_API_KEY =", os.environ.get("OPENAI_API_KEY"))
print("GROQ_API_KEY =", os.environ.get("GROQ_API_KEY"))


GROQ_API_KEY = None


In [30]:
#pip install openai
#%pip install llama-index
# %pip install pypdf
# %pip install langchain
# %pip install transformers
# %pip install pymupdf
# %pip install sentence-transformers
# %pip install faiss-cpu
# %pip install -U pypdf pymupdf
# %pip install openai llama-index langchain transformers sentence-transformers faiss-cpu
#%pip install llama-index-llms-groq
#%pip install -U llama-index-core llama-index-readers-file pypdf pymupdf
%pip install huggingface_hub[hf_xet]



   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   -------------- ------------------------- 1.0/2.9 MB 5.0 MB/s eta 0:00:01
   ------------------------- -------------- 1.8/2.9 MB 4.4 MB/s eta 0:00:01
   ------------------------------------ --- 2.6/2.9 MB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 2.9/2.9 MB 4.1 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq

Settings.llm = Groq(
    model="llama3-70b-8192",
    api_key="api key",
    base_url="https://api.openai.com/v1"
)


In [ ]:
import os
os.environ["api key"] = "sk-..."  # Apna actual key yahan daalo

from llama_index.core import VectorStoreIndex, Settings
from llama_index.llms.openai import OpenAI

llm = OpenAI(model='gpt-3.5-turbo', temperature=0.1)
Settings.llm = llm
Settings.embed_model = 'local:BAAI/bge-small-en-v1.5'

index = VectorStoreIndex.from_documents([document])
query_engine = index.as_query_engine()

2026-01-15 19:36:39,926 - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
2026-01-15 19:36:45,179 - INFO - 1 prompt is loaded, with the key: query


In [8]:
# from llama_index.core import SimpleDirectoryReader
# from llama_index.readers.file import PDFReader

# documents = SimpleDirectoryReader(
#     input_files=[
#         r"C:\Users\DELL\OneDrive\Desktop\Sarfraz\PW Data Science\Project\RAG_Application_Using_LLM\Medical_Cost_Prediction.pdf"
#     ],
#     file_extractor={
#         ".pdf": PDFReader()
#     }
# ).load_data()
# print(f"Number of documents: {len(documents)}")


import traceback
from llama_index.core import SimpleDirectoryReader
from llama_index.readers.file import PyMuPDFReader

try:
    print("Attempting to load PDF with PyMuPDFReader...")
    documents = SimpleDirectoryReader(
        input_files=[
            r"C:\Users\DELL\OneDrive\Desktop\Sarfraz\PW Data Science\Project\RAG_Application_Using_LLM\Medical_Cost_Prediction.pdf"
        ],
        file_extractor={
            ".pdf": PyMuPDFReader()
        },
        raise_on_error=True
    ).load_data()
    print(f"Number of documents: {len(documents)}")
except Exception:
    traceback.print_exc()


Attempting to load PDF with PyMuPDFReader...
Number of documents: 4


In [9]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

4 

<class 'llama_index.core.schema.Document'>
Doc ID: c4d2f10f-d942-4dc0-b359-e2747d7b5619
Text: 11/28/23, 11:00 PM README.md - Grip localhost:6419 1/4 README.md
Medical Cost Prediction Predicting medical costs of individuals based
on different features using several ML (Regression) algorithms. The
application was deployed on AWS EC2 through AWS ECR (Dockerized
Container). Dataset The Medical Cost Prediction consists of around
1300 records ...


## Basic RAG Pipeline

### Pipeline includes three components: Ingestion -> Retrieval -> Synthesis

##### **Ingestion**: Documents are divided into chunks -> chuncks are embedded using embedding model -> the embeddings are stored in a Vector Store Index
##### **Retrieval**: User's query is matched with the embeddings of chunks in index -> Top K chunks are taken out that match the embeddings of the query
##### **Synthesis**: K chunks taken in the previous component are combined with the user's query -> Combined text (embeddings) is sent to the LLM for its response

In [15]:
from rag_helpers import get_merged_document

# Load and merge your PDF in one line
document = get_merged_document(
    r"C:\Users\DELL\OneDrive\Desktop\Sarfraz\PW Data Science\Project\RAG_Application_Using_LLM\Medical_Cost_Prediction.pdf"
)
print(f"Document loaded: {len(document.text)} characters")

Document loaded: 3698 characters


In [18]:
from rag_helpers import load_pdf_documents, merge_documents

# Load documents
documents = load_pdf_documents(r"C:\Users\DELL\OneDrive\Desktop\Sarfraz\PW Data Science\Project\RAG_Application_Using_LLM\Medical_Cost_Prediction.pdf")
print(f"Loaded {len(documents)} documents")

# Merge them
document = merge_documents(documents)

Loaded 4 documents


In [ ]:
# Merging the elements in the documents above
from llama_index.core import Document

document = Document(text='\n\n'.join([doc.text for doc in documents]))

In [21]:
from rag_helpers import merge_documents

document = merge_documents(documents)

In [24]:
from rag_helpers import load_pdf_documents, merge_documents

# Load PDF documents
documents = load_pdf_documents(
    r"C:\Users\DELL\OneDrive\Desktop\Sarfraz\PW Data Science\Project\RAG_Application_Using_LLM\Medical_Cost_Prediction.pdf"
)
print(f"Number of documents: {len(documents)}")

# Merge into single document
document = merge_documents(documents)
print(f"Merged document length: {len(document.text)} characters")

Number of documents: 4
Merged document length: 3698 characters


In [31]:
from llama_index.core import VectorStoreIndex, Settings
from llama_index.llms.openai import OpenAI

# Configure LLM
llm = OpenAI(model='gpt-3.5-turbo', temperature=0.1)

# Set global settings
Settings.llm = llm
Settings.embed_model = 'local:BAAI/bge-small-en-v1.5'

# Create index
index = VectorStoreIndex.from_documents([document])

print("✅ Index created successfully!")

2026-01-15 19:32:44,950 - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
2026-01-15 19:32:50,353 - INFO - 1 prompt is loaded, with the key: query


✅ Index created successfully!


In [32]:
# Merging the elements in the documents above
from llama_index import Document
from llama_index.core import Document

document = Document(text='\n\n'.join([doc.text for doc in documents]))

ImportError: cannot import name 'Document' from 'llama_index' (unknown location)

### Ingestion

In [34]:
# Vector Store Index is used to store the chunks (from documents), text (chunk's text), and their corresponding embeddings
# Settings is used to configure the LLM and embedding model (ServiceContext is deprecated in v0.10+)
from llama_index.core import VectorStoreIndex, Settings
from llama_index.llms.openai import OpenAI

llm = OpenAI(model='gpt-3.5-turbo', temperature=0.1)

# Set global settings (NEW way - ServiceContext is deprecated)
Settings.llm = llm
Settings.embed_model = 'local:BAAI/bge-small-en-v1.5'

# Create index - Settings will be used automatically
index = VectorStoreIndex.from_documents([document])

2026-01-15 19:34:46,061 - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
2026-01-15 19:34:51,098 - INFO - 1 prompt is loaded, with the key: query


Initializing query engine from the above index that allows us to send user's query to the **synthesis** component

In [35]:
query_engine = index.as_query_engine()

In [ ]:
import os
os.environ["api key"] = "sk-..."  # Apna actual key yahan daalo

from llama_index.core import VectorStoreIndex, Settings
from llama_index.llms.openai import OpenAI

llm = OpenAI(model='gpt-3.5-turbo', temperature=0.1)
Settings.llm = llm
Settings.embed_model = 'local:BAAI/bge-small-en-v1.5'

index = VectorStoreIndex.from_documents([document])
query_engine = index.as_query_engine()

In [ ]:
import os
os.environ["GROQ_API_KEY"] = "api key"  # Apna Groq API key yahan daalo

from llama_index.core import VectorStoreIndex, Settings
from llama_index.llms.groq import Groq

Settings.llm = Groq(
    model="llama3-70b-8192",
    api_key=os.environ.get("GROQ_API_KEY")
)
Settings.embed_model = 'local:BAAI/bge-small-en-v1.5'

index = VectorStoreIndex.from_documents([document])
query_engine = index.as_query_engine()

2026-01-15 19:39:32,840 - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
2026-01-15 19:39:40,199 - INFO - 1 prompt is loaded, with the key: query


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "api key"  # Apna OpenAI API key yahan daalo

from llama_index.core import VectorStoreIndex, Settings
from llama_index.llms.openai import OpenAI

Settings.llm = OpenAI(model='gpt-3.5-turbo', temperature=0.1, api_key=os.environ.get("OPENAI_API_KEY"))
Settings.embed_model = 'local:BAAI/bge-small-en-v1.5'

index = VectorStoreIndex.from_documents([document])
query_engine = index.as_query_engine()

2026-01-15 19:39:50,377 - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
2026-01-15 19:39:54,927 - INFO - 1 prompt is loaded, with the key: query


In [ ]:
# Groq API key set karo
import os
os.environ["GROQ_API_KEY"] = "api key"

# Index banate waqt Groq LLM use karo
from llama_index.core import VectorStoreIndex, Settings
from llama_index.llms.groq import Groq

Settings.llm = Groq(
    model="llama-3.3-70b-versatile",  # LATEST working model (Jan 2026)
    api_key=os.environ.get("GROQ_API_KEY")
)
Settings.embed_model = 'local:BAAI/bge-small-en-v1.5'

# Index create karo
index = VectorStoreIndex.from_documents([document])

# Query engine banao
query_engine = index.as_query_engine()

# Ab query karo
response = query_engine.query('How is the project deployed')
print(str(response))

2026-01-15 20:24:34,701 - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
2026-01-15 20:24:38,409 - INFO - 1 prompt is loaded, with the key: query
2026-01-15 20:24:39,307 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


The project is deployed using AWS ECR and AWS EC2. To deploy, a Docker image is created and checked locally. Then, workflows are added to the GitHub repository, and the appropriate deployment method is chosen. An IAM User is created with necessary policies, and access keys are generated. An ECR repository is created, and an EC2 instance is set up with the necessary packages for Docker. The runner is set up on the EC2 instance, and secret keys are created in GitHub Actions.


In [47]:
response = query_engine.query('How is the project deployed')
print(str(response))

2026-01-15 20:24:44,513 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


The project is deployed using AWS ECR and AWS EC2. To deploy, a docker image is created and checked locally. Then, workflows are added to the GitHub repository, and the appropriate deployment method is chosen. An IAM User is created with necessary policies, and access keys are generated. A new ECR repository is created, and an EC2 instance is set up with necessary packages for docker. The runner is then set up on the EC2 instance, and secret keys are created in GitHub Actions.


### Evaluation setup using TruLens

##### TruLens is a software tool that helps you to objectively measure the quality and effectiveness of your LLM-based applications using feedback functions.

In [48]:
eval_questions = []
with open('eval_questions.txt','r') as file:
    for line in file:
        # Removing new line character and converting it to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the technologies used in the project?
What is the installation process?
Can you talk about the dataset used in the project?
Can you explain the usage of the project?
What is the process of deployment used in the project?


In [53]:
from trulens_eval import Tru, TruLlama
tru = Tru()

tru.reset_database()

2026-01-15 20:43:13,101 - INFO - Context impl SQLiteImpl.
2026-01-15 20:43:13,105 - INFO - Will assume non-transactional DDL.
Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]


In [ ]:
#%pip install trulens-eval --upgrade

  Using cached trulens_eval-2.5.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached trulens_core-2.5.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached trulens_dashboard-2.5.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached trulens_feedback-2.5.2-py3-none-any.whl.metadata (17 kB)
  Using cached alembic-1.18.1-py3-none-any.whl.metadata (7.2 kB)
  Using cached dill-0.3.9-py3-none-any.whl.metadata (10 kB)
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached opentelemetry_api-1.39.1-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_proto-1.39.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached opentelemetry_sdk-1.39.1-py3-none-any.whl.metadata (1.5 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached trulens_otel_semconv-2.5.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached markdown_it_py-4.0.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached ipywidgets-8.1.8-py3-none-any.whl.metadata (2.4 kB)
  Using cach

In [56]:
# Purana import (deprecated)
# from trulens_eval import Tru, TruLlama

# Naya import (v1.0+)
from trulens.core import Tru
from trulens.apps.llamaindex import TruLlama

tru = Tru()
tru.reset_database()

tru_recorder = TruLlama(query_engine, app_id='Direct Query Engine')

C:\Users\DELL\AppData\Local\Temp\ipykernel_4328\1258527641.py:8: DeprecationWarning: Tru is deprecated, use TruSession instead.
  tru = Tru()
Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]


instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.embeddings.multi_modal_base.MultiModalEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.base.embeddings.base.BaseEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.schema.TransformComponent'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.schema.BaseComponent'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'pydantic.main.BaseModel'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base

In [57]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [58]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [59]:
records.head()

,app_id,app_name,app_version,app_json,type,record_id,input_id,input,output,tags,...,cost_json,perf_json,ts,latency,total_tokens,total_cost,eval_cost,eval_cost_snowflake,cost_currency,num_events
0,app_hash_6e8221fde876d15698298cea8c0d1bd6,Direct Query Engine,base,"{'app_name': 'Direct Query Engine', 'app_versi...",SPAN,6b1757ba-2b9e-44d7-b20f-152b44e07907,,What are the technologies used in the project?,The technologies used in the project include M...,,...,"{'n_tokens': 0, 'cost': 0.0}","{'start_time': 2026-01-15 20:46:53.652916, 'en...",2026-01-15 20:46:53.652916,0.700009,0,0.0,0.0,0.0,USD,8
1,app_hash_6e8221fde876d15698298cea8c0d1bd6,Direct Query Engine,base,"{'app_name': 'Direct Query Engine', 'app_versi...",SPAN,3775e8dd-6d89-43a1-8e5c-84049df37800,,What is the installation process?,The installation process involves installing t...,,...,"{'n_tokens': 0, 'cost': 0.0}","{'start_time': 2026-01-15 20:46:54.352925, 'en...",2026-01-15 20:46:54.352925,0.342185,0,0.0,0.0,0.0,USD,8
2,app_hash_6e8221fde876d15698298cea8c0d1bd6,Direct Query Engine,base,"{'app_name': 'Direct Query Engine', 'app_versi...",SPAN,01a80b26-1457-479c-aaf8-421f3a106698,,Can you talk about the dataset used in the pro...,The dataset used in the project consists of ar...,,...,"{'n_tokens': 0, 'cost': 0.0}","{'start_time': 2026-01-15 20:46:54.695110, 'en...",2026-01-15 20:46:54.695110,0.599921,0,0.0,0.0,0.0,USD,8
3,app_hash_6e8221fde876d15698298cea8c0d1bd6,Direct Query Engine,base,"{'app_name': 'Direct Query Engine', 'app_versi...",SPAN,eaf05adc-80d0-407e-a78b-0be3ed15e034,,Can you explain the usage of the project?,The usage of the project involves three main s...,,...,"{'n_tokens': 0, 'cost': 0.0}","{'start_time': 2026-01-15 20:46:55.295031, 'en...",2026-01-15 20:46:55.295031,0.589804,0,0.0,0.0,0.0,USD,8
4,app_hash_6e8221fde876d15698298cea8c0d1bd6,Direct Query Engine,base,"{'app_name': 'Direct Query Engine', 'app_versi...",SPAN,e00954ed-d466-45ad-a52a-443be916418e,,What is the process of deployment used in the ...,The process of deployment used in the project ...,,...,"{'n_tokens': 0, 'cost': 0.0}","{'start_time': 2026-01-15 20:46:55.884835, 'en...",2026-01-15 20:46:55.884835,0.512862,0,0.0,0.0,0.0,USD,8


In [ ]:
# launches on http://localhost:8501/
tru.run_dashboard()

C:\Users\DELL\AppData\Local\Temp\ipykernel_4328\1997498841.py:2: DeprecationWarning: Method `run_dashboard` has been renamed or moved to `trulens.dashboard.run.run_dashboard`.

  tru.run_dashboard()


Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…